In [ ]:
import cv2
import time
from google.colab.patches import cv2_imshow
import IPython

In [ ]:
# obama_01.mp4
# haarcascade_frontalface_alt.xml

from google.colab import files
files.upload()

In [ ]:
file_name = 'obama_01.mp4'
face_cascade_name = 'haarcascade_frontalface_alt.xml'
frame_count = 0

tracker = cv2.TrackerKCF_create()

face_cascade = cv2.CascadeClassifier()
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('### Error loading face cascade ###')
    exit(0)

detected = False
frame_mode = 'Tracking'
elapsed_time = 0
trackers = cv2.MultiTracker_create()

vs = cv2.VideoCapture(file_name)

while True:
    IPython.display.clear_output(wait=True)
    ret, frame = vs.read()
    if frame is None:
        print('### No more frame ###')
        break
    start_time = time.time()
    frame_count += 1
    if detected:
        frame_mode = 'Tracking'
        (success, boxes) = trackers.update(frame)
        for box in boxes:
            (x, y, w, h) = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    else:
        frame_mode = 'Detection'
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_gray = cv2.equalizeHist(frame_gray)            
        faces = face_cascade.detectMultiScale(frame_gray)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 4)

        trackers.add(tracker, frame, tuple(faces[0]))
        detected = True

    cv2_imshow(frame)
    frame_time = time.time() - start_time
    elapsed_time += frame_time
    print("[{}] Frame {} time {}".format(frame_mode, frame_count, frame_time))

print("Elapsed time {}".format(elapsed_time))
vs.release()